# 1. 데이터 읽기 

> 날씨데이터와 배달데이터는 서로 읍면동을 표현하는 것이 매우 다르다. 예를들어 어떤 지역은 배달데이터에서 1동,2동,3동으로 표현된반면 어떤 지역은 날씨데이터에서 1동,2동,3동으로 표현된 곳도 있고, 배달데이터에서는 ~로n가로 표현했다면 날씨데이터에서는 ~제n동 이런식. 913개의 읍면동 지역의 2/3가 일치하지 않고, 다름에 일관적인 규칙이 없어서 **시군구 단위로 시간대별** 데이터를 가지고 배달데이터와 함께 시각화 하는 것으로 결정하였다.

In [ ]:
import pandas as pd
import numpy as np

## 1.1 강수형태

In [ ]:
rainSnow1 = pd.read_csv('/content/drive/MyDrive/weather/강수형태1_2019_2021.csv', parse_dates=['날짜'])
rainSnow2 = pd.read_csv('/content/drive/MyDrive/weather/강수형태2_2019_2021.csv', parse_dates=['날짜'])

In [ ]:
print(rainSnow1.info())
rainSnow2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32344512 entries, 0 to 32344511
Data columns (total 6 columns):
 #   Column  Dtype         
---  ------  -----         
 0   광역시도    object        
 1   시군구     object        
 2   읍면동     object        
 3   날짜      datetime64[ns]
 4   시간대     int64         
 5   강수형태    object        
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 1.4+ GB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31863216 entries, 0 to 31863215
Data columns (total 6 columns):
 #   Column  Dtype         
---  ------  -----         
 0   광역시도    object        
 1   시군구     object        
 2   읍면동     object        
 3   날짜      datetime64[ns]
 4   시간대     int64         
 5   강수형태    object        
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 1.4+ GB


In [ ]:
rainSnow2

,광역시도,시군구,읍면동,날짜,시간대,강수형태
0,서울특별시,강남구,삼성1동,2021-08-01,0,없음
1,서울특별시,강남구,삼성1동,2021-08-01,1,없음
2,서울특별시,강남구,삼성1동,2021-08-01,2,없음
3,서울특별시,강남구,삼성1동,2021-08-01,3,비
4,서울특별시,강남구,삼성1동,2021-08-01,4,없음
...,...,...,...,...,...,...
31863211,서울특별시,관악구,대학동,2021-08-31,19,없음
31863212,서울특별시,관악구,대학동,2021-08-31,20,없음
31863213,서울특별시,관악구,대학동,2021-08-31,21,없음
31863214,서울특별시,관악구,대학동,2021-08-31,22,없음


In [ ]:
print(rainSnow1.isna().sum(),"\n")
rainSnow2.isna().sum()

광역시도        0
시군구         0
읍면동         0
날짜          0
시간대         0
강수형태    37980
dtype: int64 



광역시도        0
시군구         0
읍면동         0
날짜          0
시간대         0
강수형태    37481
dtype: int64

In [ ]:
rainSnow1[rainSnow1['강수형태'].isna()]

,광역시도,시군구,읍면동,날짜,시간대,강수형태
220,충청남도,천안시서북구,성정2동,2021-08-10,4,NaN
957,서울특별시,송파구,가락본동,2021-08-09,21,NaN
964,서울특별시,송파구,가락본동,2021-08-10,4,NaN
8439,인천광역시,연수구,옥련2동,2021-05-17,15,NaN
8501,인천광역시,연수구,옥련2동,2021-05-20,5,NaN
...,...,...,...,...,...,...
32333063,충청북도,단양군,대강면,2020-04-11,23,NaN
32335769,전라남도,영광군,군서면,2019-08-02,17,NaN
32341370,전라남도,영광군,군서면,2020-03-23,2,NaN
32341837,전라남도,영광군,군서면,2020-04-11,13,NaN


### 1.1.1 데이터 정제

날씨데이터에 지역을 붙여주는 과정에서 크롤러를 이용해 데이터를 수집하면서 중복으로 다운받은 데이터가 있는것을 알게되었다. `drop_duplicates` 을 이용하여 중복을 제거한다.

In [ ]:
rainSnow1.drop_duplicates(inplace=True)
rainSnow2.drop_duplicates(inplace=True)

### 1.1.2 시군구별 groupby

강수형태는 평균을 내거나 코드값의 형태라도 최대값을 낼 수 없으므로 **최빈값**으로 선택하였다. 같은 빈도의 여러 최빈값이 있으면, data에서 처음 발견된 첫 번째 값을 반환. **특히 `pd.Series.mode`는 최빈값이 여러개이면 몽땅 리스트로 반환하기 때문에 꼭 인덱스를 명시해야 하나만 가져올 수 있다!** 

> 최빈값 참고 : https://newbedev.com/groupby-pandas-dataframe-and-select-most-common-value

In [ ]:
# 코드형태(int)로 다시 바꾼다. 참고링크를 보면 string형태라도 최빈값이 잘 구해지는 것 같은데, 나는 왜인지 에러가 나서 다시 코드형태로 매핑하였다.
str2code =  { np.nan : -1, '없음' :0  ,  '비':1,  '비/눈': 2, '눈': 3 , '소나기' : 4 } 
rainSnow1['강수형태'] = rainSnow1['강수형태'].map(str2code) 
rainSnow2['강수형태'] = rainSnow2['강수형태'].map(str2code) 
rainSnow1

,광역시도,시군구,읍면동,날짜,시간대,강수형태
0,충청남도,천안시서북구,성정2동,2021-08-01,0,0
1,충청남도,천안시서북구,성정2동,2021-08-01,1,0
2,충청남도,천안시서북구,성정2동,2021-08-01,2,0
3,충청남도,천안시서북구,성정2동,2021-08-01,3,0
4,충청남도,천안시서북구,성정2동,2021-08-01,4,0
...,...,...,...,...,...,...
32344507,전라남도,영광군,군서면,2020-07-31,19,0
32344508,전라남도,영광군,군서면,2020-07-31,20,0
32344509,전라남도,영광군,군서면,2020-07-31,21,0
32344510,전라남도,영광군,군서면,2020-07-31,22,0


In [ ]:
rainSnow1[rainSnow1['강수형태']==-1]

,광역시도,시군구,읍면동,날짜,시간대,강수형태
220,충청남도,천안시서북구,성정2동,2021-08-10,4,-1
957,서울특별시,송파구,가락본동,2021-08-09,21,-1
964,서울특별시,송파구,가락본동,2021-08-10,4,-1
8439,인천광역시,연수구,옥련2동,2021-05-17,15,-1
8501,인천광역시,연수구,옥련2동,2021-05-20,5,-1
...,...,...,...,...,...,...
32333063,충청북도,단양군,대강면,2020-04-11,23,-1
32335769,전라남도,영광군,군서면,2019-08-02,17,-1
32341370,전라남도,영광군,군서면,2020-03-23,2,-1
32341837,전라남도,영광군,군서면,2020-04-11,13,-1


In [ ]:
#강수형태는 최빈값으로 
rainSnow1_new = rainSnow1.groupby(['광역시도','시군구','날짜','시간대'])['강수형태'].agg(
    lambda x: pd.Series.mode(x)[0]).reset_index()
rainSnow1_new

,광역시도,시군구,날짜,시간대,강수형태
0,강원도,강릉시,2019-08-01,0,0
1,강원도,강릉시,2019-08-01,1,0
2,강원도,강릉시,2019-08-01,2,0
3,강원도,강릉시,2019-08-01,3,0
4,강원도,강릉시,2019-08-01,4,0
...,...,...,...,...,...
4599043,충청북도,충주시,2021-08-31,19,1
4599044,충청북도,충주시,2021-08-31,20,1
4599045,충청북도,충주시,2021-08-31,21,1
4599046,충청북도,충주시,2021-08-31,22,1


In [ ]:
rainSnow1_new['강수형태'].unique() #잘 구해졌다!

array([ 0, -1,  1,  2,  3,  4])

In [ ]:
#배달데이터는 세종특별자치시의 시군구가 존재하지 않음
rainSnow1_new['시군구'] = np.where(rainSnow1_new['시군구']=='세종특별자치시', '\\N', rainSnow1_new['시군구'])
# 창원시를 제외한 ~시~구 형태의 시군구는 ~시 ~구로 변환(띄어쓰기 필요)
rainSnow1_new['시군구'] = np.where(rainSnow1_new['시군구'].str.endswith('구') & rainSnow1_new['시군구'].str.contains('시') & ~rainSnow1_new['시군구'].str.startswith('창원'),
                        rainSnow1_new['시군구'].str.replace('시',"시 "), rainSnow1_new['시군구'])

In [ ]:
rainSnow1_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4599048 entries, 0 to 4599047
Data columns (total 5 columns):
 #   Column  Dtype         
---  ------  -----         
 0   광역시도    object        
 1   시군구     object        
 2   날짜      datetime64[ns]
 3   시간대     int64         
 4   강수형태    object        
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 175.4+ MB


In [ ]:
#백업해두기
rainSnow1_new.to_csv('/content/drive/MyDrive/시군구_시간대별_강수형태1.csv',index=False, encoding="utf-8-sig") 

In [ ]:
rainSnow2[rainSnow2['강수형태']==-1]

,광역시도,시군구,읍면동,날짜,시간대,강수형태
213,서울특별시,강남구,삼성1동,2021-08-09,21,-1
220,서울특별시,강남구,삼성1동,2021-08-10,4,-1
957,전라남도,신안군,암태면,2021-08-09,21,-1
964,전라남도,신안군,암태면,2021-08-10,4,-1
1708,경상북도,의성군,신평면,2021-08-10,4,-1
...,...,...,...,...,...,...
31859330,충청북도,단양군,단성면,2020-03-23,2,-1
31859807,충청북도,단양군,단성면,2020-04-11,23,-1
31862424,충청북도,단양군,단성면,2020-07-30,0,-1
31862685,서울특별시,관악구,대학동,2021-08-09,21,-1


In [ ]:
rainSnow2_new = rainSnow2.groupby(['광역시도','시군구','날짜','시간대'])['강수형태'].agg(
    lambda x: pd.Series.mode(x)[0]).reset_index()
rainSnow2_new

,광역시도,시군구,날짜,시간대,강수형태
0,강원도,강릉시,2019-08-01,0,0
1,강원도,강릉시,2019-08-01,1,0
2,강원도,강릉시,2019-08-01,2,0
3,강원도,강릉시,2019-08-01,3,0
4,강원도,강릉시,2019-08-01,4,0
...,...,...,...,...,...
4606339,충청북도,충주시,2021-08-31,19,1
4606340,충청북도,충주시,2021-08-31,20,1
4606341,충청북도,충주시,2021-08-31,21,1
4606342,충청북도,충주시,2021-08-31,22,1


In [ ]:
rainSnow2_new['강수형태'].unique() #잘 구해졌다!

array([ 0, -1,  1,  2,  3,  4])

In [ ]:
#배달데이터는 세종특별자치시의 시군구가 존재하지 않음
rainSnow2_new['시군구'] = np.where(rainSnow2_new['시군구']=='세종특별자치시', '\\N', rainSnow2_new['시군구'])
# 창원시를 제외한 ~시~구 형태의 시군구는 ~시 ~구로 변환(띄어쓰기 필요)
rainSnow2_new['시군구'] = np.where(rainSnow2_new['시군구'].str.endswith('구') & rainSnow2_new['시군구'].str.contains('시') & ~rainSnow2_new['시군구'].str.startswith('창원'),
                        rainSnow2_new['시군구'].str.replace('시',"시 "), rainSnow2_new['시군구'])

In [ ]:
rainSnow2_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4606344 entries, 0 to 4606343
Data columns (total 5 columns):
 #   Column  Dtype         
---  ------  -----         
 0   광역시도    object        
 1   시군구     object        
 2   날짜      datetime64[ns]
 3   시간대     int64         
 4   강수형태    int64         
dtypes: datetime64[ns](1), int64(2), object(2)
memory usage: 175.7+ MB


In [ ]:
#백업해두기
rainSnow2_new.to_csv('/content/drive/MyDrive/시군구_시간대별_강수형태2.csv',index=False, encoding="utf-8-sig")

In [ ]:
rainSnow = pd.concat([rainSnow1_new,rainSnow2_new], ignore_index=True)
rainSnow

,광역시도,시군구,날짜,시간대,강수형태
0,강원도,강릉시,2019-08-01,0,0
1,강원도,강릉시,2019-08-01,1,0
2,강원도,강릉시,2019-08-01,2,0
3,강원도,강릉시,2019-08-01,3,0
4,강원도,강릉시,2019-08-01,4,0
...,...,...,...,...,...
9205387,충청북도,충주시,2021-08-31,19,1
9205388,충청북도,충주시,2021-08-31,20,1
9205389,충청북도,충주시,2021-08-31,21,1
9205390,충청북도,충주시,2021-08-31,22,1


In [ ]:
rainSnow_new = rainSnow.groupby(['광역시도','시군구','날짜','시간대'])['강수형태'].agg(
    lambda x: pd.Series.mode(x)[0]).reset_index()
rainSnow_new

,광역시도,시군구,날짜,시간대,강수형태
0,강원도,강릉시,2019-08-01,0,0
1,강원도,강릉시,2019-08-01,1,0
2,강원도,강릉시,2019-08-01,2,0
3,강원도,강릉시,2019-08-01,3,0
4,강원도,강릉시,2019-08-01,4,0
...,...,...,...,...,...
4626859,충청북도,충주시,2021-08-31,19,1
4626860,충청북도,충주시,2021-08-31,20,1
4626861,충청북도,충주시,2021-08-31,21,1
4626862,충청북도,충주시,2021-08-31,22,1


In [ ]:
#코드형태에서 다시 눈, 비 형태로 전환
code2str =  { -1 : np.nan, 0 : '없음', 1: '비', 2: '비/눈', 3: '눈', 4: '소나기'} 
rainSnow_new['강수형태'] = rainSnow_new['강수형태'].map(code2str) 
rainSnow_new

,광역시도,시군구,날짜,시간대,강수형태
0,강원도,강릉시,2019-08-01,0,없음
1,강원도,강릉시,2019-08-01,1,없음
2,강원도,강릉시,2019-08-01,2,없음
3,강원도,강릉시,2019-08-01,3,없음
4,강원도,강릉시,2019-08-01,4,없음
...,...,...,...,...,...
4626859,충청북도,충주시,2021-08-31,19,비
4626860,충청북도,충주시,2021-08-31,20,비
4626861,충청북도,충주시,2021-08-31,21,비
4626862,충청북도,충주시,2021-08-31,22,비


In [ ]:
rainSnow_new['강수형태'].unique() #잘 구해졌다!

array(['없음', nan, '비', '비/눈', '눈', '소나기'], dtype=object)

In [ ]:
#최종본 저장
rainSnow_new.to_csv('/content/drive/MyDrive/시군구_시간대별_최빈강수형태.csv',index=False, encoding="utf-8-sig")

In [ ]:
del rainSnow_new, rainSnow1, rainSnow2

## 1.2 강수

In [ ]:
rain1 = pd.read_csv('/content/drive/MyDrive/weather/강수1_2019_2021.csv', parse_dates=['날짜'])
rain2 = pd.read_csv('/content/drive/MyDrive/weather/강수2_2019_2021.csv', parse_dates=['날짜'])

In [ ]:
rain1

,광역시도,시군구,읍면동,날짜,시간대,강수
0,경상남도,통영시,도산면,2020-08-01,0,0.0
1,경상남도,통영시,도산면,2020-08-01,1,0.0
2,경상남도,통영시,도산면,2020-08-01,2,0.0
3,경상남도,통영시,도산면,2020-08-01,3,0.0
4,경상남도,통영시,도산면,2020-08-01,4,0.0
...,...,...,...,...,...,...
32288947,경기도,고양시덕양구,고양동,2021-08-31,19,0.0
32288948,경기도,고양시덕양구,고양동,2021-08-31,20,0.0
32288949,경기도,고양시덕양구,고양동,2021-08-31,21,0.0
32288950,경기도,고양시덕양구,고양동,2021-08-31,22,0.0


In [ ]:
rain2

,광역시도,시군구,읍면동,날짜,시간대,강수
0,경상북도,안동시,녹전면,2019-08-01,0,0.0
1,경상북도,안동시,녹전면,2019-08-01,1,0.0
2,경상북도,안동시,녹전면,2019-08-01,2,0.0
3,경상북도,안동시,녹전면,2019-08-01,3,0.0
4,경상북도,안동시,녹전면,2019-08-01,4,0.0
...,...,...,...,...,...,...
31927531,대구광역시,중구,성내2동,2020-07-31,19,0.0
31927532,대구광역시,중구,성내2동,2020-07-31,20,0.0
31927533,대구광역시,중구,성내2동,2020-07-31,21,0.0
31927534,대구광역시,중구,성내2동,2020-07-31,22,0.0


In [ ]:
print(rain1.info())
rain2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32288952 entries, 0 to 32288951
Data columns (total 6 columns):
 #   Column  Dtype         
---  ------  -----         
 0   광역시도    object        
 1   시군구     object        
 2   읍면동     object        
 3   날짜      datetime64[ns]
 4   시간대     int64         
 5   강수      float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(3)
memory usage: 1.4+ GB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31927536 entries, 0 to 31927535
Data columns (total 6 columns):
 #   Column  Dtype         
---  ------  -----         
 0   광역시도    object        
 1   시군구     object        
 2   읍면동     object        
 3   날짜      datetime64[ns]
 4   시간대     int64         
 5   강수      float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(3)
memory usage: 1.4+ GB


In [ ]:
print(rain1.isna().sum())
rain1.isna().sum()

광역시도    0
시군구     0
읍면동     0
날짜      0
시간대     0
강수      0
dtype: int64


광역시도    0
시군구     0
읍면동     0
날짜      0
시간대     0
강수      0
dtype: int64

### 1.2.1 데이터 정제

In [ ]:
rain1.drop_duplicates(inplace=True)
rain2.drop_duplicates(inplace=True)

#### ⭐결측치 처리

강수가 **-1**인것은 missing이다. 강수형태의 NaN과 같다. (기상자료 개방포털 포멧 참고) </br>
현재 결측치가 **-1**로 처리되어 있기 때문에 평균등을 계산하는데 매우 큰 영향을 미친다. **결측치를 np.nan으로 바꾼다.** 

In [ ]:
rain1.describe()

,시간대,강수
count,3.227064e+07,3.227064e+07
mean,1.150000e+01,1.734485e-01
std,6.922187e+00,1.291389e+00
min,0.000000e+00,-1.000000e+00
25%,5.750000e+00,0.000000e+00
50%,1.150000e+01,0.000000e+00
75%,1.725000e+01,0.000000e+00
max,2.300000e+01,1.000000e+02


In [ ]:
rain2.describe()

,시간대,강수
count,3.191878e+07,3.191878e+07
mean,1.150000e+01,1.733980e-01
std,6.922187e+00,1.289811e+00
min,0.000000e+00,-1.000000e+00
25%,5.750000e+00,0.000000e+00
50%,1.150000e+01,0.000000e+00
75%,1.725000e+01,0.000000e+00
max,2.300000e+01,1.000000e+02


In [ ]:
rain1['강수'] = np.where(rain1['강수']<0, np.nan, rain1['강수'])
rain2['강수'] = np.where(rain2['강수']<0, np.nan, rain2['강수'])

In [ ]:
rain1.describe()

,시간대,강수
count,3.227064e+07,3.223275e+07
mean,1.150000e+01,1.748276e-01
std,6.922187e+00,1.291520e+00
min,0.000000e+00,0.000000e+00
25%,5.750000e+00,0.000000e+00
50%,1.150000e+01,0.000000e+00
75%,1.725000e+01,0.000000e+00
max,2.300000e+01,1.000000e+02


In [ ]:
rain2.describe()

,시간대,강수
count,3.191878e+07,3.188117e+07
mean,1.150000e+01,1.747817e-01
std,6.922187e+00,1.289942e+00
min,0.000000e+00,0.000000e+00
25%,5.750000e+00,0.000000e+00
50%,1.150000e+01,0.000000e+00
75%,1.725000e+01,0.000000e+00
max,2.300000e+01,1.000000e+02


### 1.2.2 시군구별 groupby

시군구별 그룹으로 묶을 때 **강수형태를 최빈값**을 하였기 때문에 강수의 데이터의 분포를 살펴보았을 때, 평균을 내는 것 보다 **중간값**을 가져가는 것이 강수형태가 없음일때 강수가 0이 아닌 오류를 피할 수 있을 것 같다.

In [ ]:
rain1.describe()

,시간대,강수
count,3.227064e+07,3.223275e+07
mean,1.150000e+01,1.748276e-01
std,6.922187e+00,1.291520e+00
min,0.000000e+00,0.000000e+00
25%,5.750000e+00,0.000000e+00
50%,1.150000e+01,0.000000e+00
75%,1.725000e+01,0.000000e+00
max,2.300000e+01,1.000000e+02


In [ ]:
rain2.describe()

,시간대,강수
count,3.191878e+07,3.188117e+07
mean,1.150000e+01,1.747817e-01
std,6.922187e+00,1.289942e+00
min,0.000000e+00,0.000000e+00
25%,5.750000e+00,0.000000e+00
50%,1.150000e+01,0.000000e+00
75%,1.725000e+01,0.000000e+00
max,2.300000e+01,1.000000e+02


In [ ]:
rain1 = rain1.groupby(['광역시도','시군구','날짜','시간대'])['강수'].median().reset_index()
rain1

,광역시도,시군구,날짜,시간대,강수
0,강원도,강릉시,2019-08-01,0,0.0
1,강원도,강릉시,2019-08-01,1,0.0
2,강원도,강릉시,2019-08-01,2,0.0
3,강원도,강릉시,2019-08-01,3,0.0
4,강원도,강릉시,2019-08-01,4,0.0
...,...,...,...,...,...
4573459,충청북도,충주시,2021-08-31,19,13.1
4573460,충청북도,충주시,2021-08-31,20,21.0
4573461,충청북도,충주시,2021-08-31,21,11.3
4573462,충청북도,충주시,2021-08-31,22,16.1


In [ ]:
rain2 = rain2.groupby(['광역시도','시군구','날짜','시간대'])['강수'].median().reset_index()
rain2

,광역시도,시군구,날짜,시간대,강수
0,강원도,강릉시,2019-08-01,0,0.00
1,강원도,강릉시,2019-08-01,1,0.00
2,강원도,강릉시,2019-08-01,2,0.00
3,강원도,강릉시,2019-08-01,3,0.00
4,강원도,강릉시,2019-08-01,4,0.00
...,...,...,...,...,...
4607107,충청북도,충주시,2021-08-31,19,13.10
4607108,충청북도,충주시,2021-08-31,20,21.00
4607109,충청북도,충주시,2021-08-31,21,11.30
4607110,충청북도,충주시,2021-08-31,22,15.80


강수에 음수가 있으면 안된다!

In [ ]:
rain1.describe()

,시간대,강수
count,4.573464e+06,4.569447e+06
mean,1.150000e+01,1.693257e-01
std,6.922187e+00,1.234382e+00
min,0.000000e+00,0.000000e+00
25%,5.750000e+00,0.000000e+00
50%,1.150000e+01,0.000000e+00
75%,1.725000e+01,0.000000e+00
max,2.300000e+01,8.950000e+01


In [ ]:
rain2.describe()

,시간대,강수
count,4.607112e+06,4.603123e+06
mean,1.150000e+01,1.698647e-01
std,6.922187e+00,1.239667e+00
min,0.000000e+00,0.000000e+00
25%,5.750000e+00,0.000000e+00
50%,1.150000e+01,0.000000e+00
75%,1.725000e+01,0.000000e+00
max,2.300000e+01,9.700000e+01


In [ ]:
rain = pd.concat([rain1,rain2], ignore_index=True)
rain

,광역시도,시군구,날짜,시간대,강수
0,강원도,강릉시,2019-08-01,0,0.00
1,강원도,강릉시,2019-08-01,1,0.00
2,강원도,강릉시,2019-08-01,2,0.00
3,강원도,강릉시,2019-08-01,3,0.00
4,강원도,강릉시,2019-08-01,4,0.00
...,...,...,...,...,...
9180571,충청북도,충주시,2021-08-31,19,13.10
9180572,충청북도,충주시,2021-08-31,20,21.00
9180573,충청북도,충주시,2021-08-31,21,11.30
9180574,충청북도,충주시,2021-08-31,22,15.80


In [ ]:
rain = rain.groupby(['광역시도','시군구','날짜','시간대'])['강수'].median().reset_index()
rain

,광역시도,시군구,날짜,시간대,강수
0,강원도,강릉시,2019-08-01,0,0.000
1,강원도,강릉시,2019-08-01,1,0.000
2,강원도,강릉시,2019-08-01,2,0.000
3,강원도,강릉시,2019-08-01,3,0.000
4,강원도,강릉시,2019-08-01,4,0.000
...,...,...,...,...,...
4626859,충청북도,충주시,2021-08-31,19,13.100
4626860,충청북도,충주시,2021-08-31,20,21.000
4626861,충청북도,충주시,2021-08-31,21,11.300
4626862,충청북도,충주시,2021-08-31,22,15.950


In [ ]:
#배달데이터는 세종특별자치시의 시군구가 존재하지 않음
rain['시군구'] = np.where(rain['시군구']=='세종특별자치시', '\\N', rain['시군구'])
# 창원시를 제외한 ~시~구 형태의 시군구는 ~시 ~구로 변환(띄어쓰기 필요)
rain['시군구'] = np.where(rain['시군구'].str.endswith('구') & rain['시군구'].str.contains('시') & ~rain['시군구'].str.startswith('창원'),
                        rain['시군구'].str.replace('시',"시 "), rain['시군구'])
rain[rain['시군구']=='천안시 서북구']

,광역시도,시군구,날짜,시간대,강수
4297680,충청남도,천안시 서북구,2019-08-01,0,0.0
4297681,충청남도,천안시 서북구,2019-08-01,1,0.0
4297682,충청남도,천안시 서북구,2019-08-01,2,0.0
4297683,충청남도,천안시 서북구,2019-08-01,3,0.0
4297684,충청남도,천안시 서북구,2019-08-01,4,0.0
...,...,...,...,...,...
4315963,충청남도,천안시 서북구,2021-08-31,19,15.0
4315964,충청남도,천안시 서북구,2021-08-31,20,14.0
4315965,충청남도,천안시 서북구,2021-08-31,21,8.0
4315966,충청남도,천안시 서북구,2021-08-31,22,8.5


In [ ]:
rain.describe()

,시간대,강수
count,4.626864e+06,4.623021e+06
mean,1.150000e+01,1.696626e-01
std,6.922187e+00,1.218501e+00
min,0.000000e+00,0.000000e+00
25%,5.750000e+00,0.000000e+00
50%,1.150000e+01,0.000000e+00
75%,1.725000e+01,0.000000e+00
max,2.300000e+01,8.950000e+01


In [ ]:
rain.to_csv('/content/drive/MyDrive/시군구_시간대별_중간강수.csv',index=False, encoding="utf-8-sig")

## 1.3 기온

In [ ]:
# !unzip -O cp949 '/content/drive/MyDrive/weather/기온1_2019_2021.zip' -d '/content/drive/MyDrive/weather/'

Archive:  /content/drive/MyDrive/weather/기온1_2019_2021.zip
  inflating: /content/drive/MyDrive/weather/기온1_2019_2021.csv  
  inflating: /content/drive/MyDrive/weather/기온2_2019_2021.csv  


In [ ]:
temp1 = pd.read_csv('/content/drive/MyDrive/weather/기온1_2019_2021.csv', parse_dates=['날짜'])
temp2 = pd.read_csv('/content/drive/MyDrive/weather/기온2_2019_2021.csv', parse_dates=['날짜'])

### 1.3.1. 데이터 정제

중복 제거

In [ ]:
temp1.drop_duplicates(inplace=True)
temp2.drop_duplicates(inplace=True)

In [ ]:
temp1

,광역시도,시군구,읍면동,날짜,시간대,기온
0,경상남도,김해시,장유3동,2019-08-01,0,32.200001
1,경상남도,김해시,장유3동,2019-08-01,1,32.400002
2,경상남도,김해시,장유3동,2019-08-01,2,33.200001
3,경상남도,김해시,장유3동,2019-08-01,3,33.000000
4,경상남도,김해시,장유3동,2019-08-01,4,32.400002
...,...,...,...,...,...,...
32056795,경상남도,사천시,정동면,2020-07-31,19,23.600000
32056796,경상남도,사천시,정동면,2020-07-31,20,23.600000
32056797,경상남도,사천시,정동면,2020-07-31,21,24.600000
32056798,경상남도,사천시,정동면,2020-07-31,22,26.100000


In [ ]:
temp2

,광역시도,시군구,읍면동,날짜,시간대,기온
0,충청남도,천안시동남구,원성1동,2019-08-01,0,28.799999
1,충청남도,천안시동남구,원성1동,2019-08-01,1,30.700001
2,충청남도,천안시동남구,원성1동,2019-08-01,2,31.200001
3,충청남도,천안시동남구,원성1동,2019-08-01,3,30.299999
4,충청남도,천안시동남구,원성1동,2019-08-01,4,32.099998
...,...,...,...,...,...,...
32160427,서울특별시,용산구,한강로동,2021-07-31,19,27.799999
32160428,서울특별시,용산구,한강로동,2021-07-31,20,27.700001
32160429,서울특별시,용산구,한강로동,2021-07-31,21,27.900000
32160430,서울특별시,용산구,한강로동,2021-07-31,22,28.100000


In [ ]:
temp1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32030496 entries, 0 to 32056799
Data columns (total 6 columns):
 #   Column  Dtype         
---  ------  -----         
 0   광역시도    object        
 1   시군구     object        
 2   읍면동     object        
 3   날짜      datetime64[ns]
 4   시간대     int64         
 5   기온      float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(3)
memory usage: 1.7+ GB


In [ ]:
temp2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32142120 entries, 0 to 32160431
Data columns (total 6 columns):
 #   Column  Dtype         
---  ------  -----         
 0   광역시도    object        
 1   시군구     object        
 2   읍면동     object        
 3   날짜      datetime64[ns]
 4   시간대     int64         
 5   기온      float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(3)
memory usage: 1.7+ GB


#### ⭐결측치 및 이상값 처리

기온은 시군구, 시간대별로 그룹을 지어 평균을 낼 것이기 때문에 missing을 뜻하는 **-50**은 작은 그룹의 평균을 계산하는데 매우 큰 영향을 준다. **결측치**는 전체 데이터에 비해 적은 양을 차지하고 있으므로 **np.nan으로 바꾼다**

> 30년 평년 기온 참고 https://www.weather.go.kr/weather/climate/average_30years.jsp?yy_st=2011&stn=0&norm=D&obs=0&mm=4&dd=9&x=12&y=10 


In [ ]:
temp1.describe()

,시간대,기온
count,3.203050e+07,3.203050e+07
mean,1.150000e+01,1.395124e+01
std,6.922187e+00,1.029461e+01
min,0.000000e+00,-5.000000e+01
25%,5.750000e+00,6.100000e+00
50%,1.150000e+01,1.490000e+01
75%,1.725000e+01,2.240000e+01
max,2.300000e+01,4.260000e+01


In [ ]:
temp2.describe()

,시간대,기온
count,3.214212e+07,3.214212e+07
mean,1.150000e+01,1.396459e+01
std,6.922187e+00,1.030039e+01
min,0.000000e+00,-5.000000e+01
25%,5.750000e+00,6.100000e+00
50%,1.150000e+01,1.490000e+01
75%,1.725000e+01,2.240000e+01
max,2.300000e+01,4.260000e+01


In [ ]:
temp2[temp2['기온']<-45]

,광역시도,시군구,읍면동,날짜,시간대,기온
41,충청남도,천안시동남구,원성1동,2019-08-02,17,-50.0
855,충청남도,천안시동남구,원성1동,2019-09-05,15,-50.0
861,충청남도,천안시동남구,원성1동,2019-09-05,21,-50.0
1389,충청남도,천안시동남구,원성1동,2019-09-27,21,-50.0
4928,충청남도,천안시동남구,원성1동,2020-02-22,8,-50.0
...,...,...,...,...,...,...
32158960,서울특별시,용산구,한강로동,2021-05-31,16,-50.0
32159021,서울특별시,용산구,한강로동,2021-06-03,5,-50.0
32159406,서울특별시,용산구,한강로동,2021-06-19,6,-50.0
32159720,서울특별시,용산구,한강로동,2021-07-02,8,-50.0


In [ ]:
temp1['기온'] = np.where(temp1['기온']<-30, np.nan, temp1['기온'])
temp2['기온'] = np.where(temp2['기온']<-30, np.nan, temp2['기온'])

In [ ]:
temp1[temp1['기온']<-30]

,광역시도,시군구,읍면동,날짜,시간대,기온


In [ ]:
temp2[temp2['기온']<-30]

,광역시도,시군구,읍면동,날짜,시간대,기온


그 외에도 평년 30년 기록을 참고하여 5~10월사이 매우 낮은 온도를 기록한 곳을 살펴본다.

In [ ]:
temp1[(temp1['기온']<-10) & (temp1['날짜'].dt.month >=5) & (temp1['날짜'].dt.month <=10)]

,광역시도,시군구,읍면동,날짜,시간대,기온
4378732,울산광역시,북구,강동동,2020-09-21,4,-23.1
8274121,경상북도,영덕군,창수면,2021-05-26,1,-10.3
24352801,경상북도,영덕군,축산면,2021-05-26,1,-10.3


In [ ]:
temp2[(temp2['기온']<-10) & (temp2['날짜'].dt.month >=5) & (temp2['날짜'].dt.month <=11)]

,광역시도,시군구,읍면동,날짜,시간대,기온
759188,경기도,포천시,신북면,2020-11-30,20,-10.2
759189,경기도,포천시,신북면,2020-11-30,21,-10.2
855116,경기도,포천시,영중면,2020-11-30,20,-10.2
855117,경기도,포천시,영중면,2020-11-30,21,-10.2
1640659,강원도,화천군,상서면,2020-11-29,19,-10.1
...,...,...,...,...,...,...
31838610,강원도,횡성군,우천면,2020-11-30,18,-10.2
31838611,강원도,횡성군,우천면,2020-11-30,19,-10.5
31838612,강원도,횡성군,우천면,2020-11-30,20,-10.9
31838613,강원도,횡성군,우천면,2020-11-30,21,-11.3


실제로 병곡면의 2021년 5월 26일 데이터 중 영하를 기록했던 시간대는 존재하지 않는다. **잘못기록된 값**이다.

In [ ]:
temp2[(temp2['날짜']=='2021-05-26') & (temp2['읍면동']=='병곡면') ]

,광역시도,시군구,읍면동,날짜,시간대,기온
6060120,경상북도,영덕군,병곡면,2021-05-26,0,17.600000
6060121,경상북도,영덕군,병곡면,2021-05-26,1,-10.300000
6060122,경상북도,영덕군,병곡면,2021-05-26,2,19.299999
6060123,경상북도,영덕군,병곡면,2021-05-26,3,19.200001
6060124,경상북도,영덕군,병곡면,2021-05-26,4,19.500000
6060125,경상북도,영덕군,병곡면,2021-05-26,5,21.400000
6060126,경상북도,영덕군,병곡면,2021-05-26,6,21.000000
6060127,경상북도,영덕군,병곡면,2021-05-26,7,20.299999
6060128,경상북도,영덕군,병곡면,2021-05-26,8,19.299999
6060129,경상북도,영덕군,병곡면,2021-05-26,9,18.500000


In [ ]:
temp1['기온'] = np.where((temp1['기온']<-10) & (temp1['날짜'].dt.month >=5) & (temp1['날짜'].dt.month <=10), np.nan, temp1['기온'])
temp2['기온'] = np.where((temp2['기온']<-10) & (temp2['날짜'].dt.month >=5) & (temp2['날짜'].dt.month <=10), np.nan, temp2['기온'])

영하 5도 이하인 경우도 시간대가 연속적이지 않아 이상값으로 판단하여 nan으로 바꾸었다.

In [ ]:
temp1[(temp1['기온']<-5) & (temp1['날짜'].dt.month >=5) & (temp1['날짜'].dt.month <=10)]

,광역시도,시군구,읍면동,날짜,시간대,기온
5110708,인천광역시,강화군,송해면,2020-05-11,4,-9.8
6808440,강원도,강릉시,강동면,2020-06-16,0,-5.1
8892748,인천광역시,강화군,불은면,2020-05-11,4,-9.8
10986004,인천광역시,강화군,삼산면,2020-05-11,4,-9.8
14935132,인천광역시,강화군,내가면,2020-05-11,4,-9.8
17128684,대구광역시,달성군,현풍읍,2020-06-20,4,-8.1
20205816,강원도,강릉시,옥천동,2020-06-16,0,-5.1
22028592,강원도,강릉시,송정동,2020-06-16,0,-5.1
22386288,강원도,강릉시,성산면,2020-06-16,0,-5.1
23184336,강원도,강릉시,강남동,2020-06-16,0,-5.1


In [ ]:
temp1[(temp1['날짜']=='2020-05-11') & (temp1['읍면동']=='불은면')]

,광역시도,시군구,읍면동,날짜,시간대,기온
8892744,인천광역시,강화군,불은면,2020-05-11,0,16.900000
8892745,인천광역시,강화군,불은면,2020-05-11,1,17.200001
8892746,인천광역시,강화군,불은면,2020-05-11,2,16.900000
8892747,인천광역시,강화군,불은면,2020-05-11,3,18.000000
8892748,인천광역시,강화군,불은면,2020-05-11,4,-9.800000
8892749,인천광역시,강화군,불은면,2020-05-11,5,18.100000
8892750,인천광역시,강화군,불은면,2020-05-11,6,17.299999
8892751,인천광역시,강화군,불은면,2020-05-11,7,16.200001
8892752,인천광역시,강화군,불은면,2020-05-11,8,14.900000
8892753,인천광역시,강화군,불은면,2020-05-11,9,14.600000


In [ ]:
temp1['기온'] = np.where((temp1['기온']<-5) & (temp1['날짜'].dt.month >=5) & (temp1['날짜'].dt.month <=10), np.nan, temp1['기온'])
temp2['기온'] = np.where((temp2['기온']<-5) & (temp2['날짜'].dt.month >=5) & (temp2['날짜'].dt.month <=10), np.nan, temp2['기온'])

In [ ]:
temp1.describe()

,시간대,기온
count,3.203050e+07,3.199508e+07
mean,1.150000e+01,1.402201e+01
std,6.922187e+00,1.007799e+01
min,0.000000e+00,-2.830000e+01
25%,5.750000e+00,6.100000e+00
50%,1.150000e+01,1.490000e+01
75%,1.725000e+01,2.240000e+01
max,2.300000e+01,4.260000e+01


In [ ]:
temp2.describe()

,시간대,기온
count,3.214212e+07,3.210595e+07
mean,1.150000e+01,1.403662e+01
std,6.922187e+00,1.007995e+01
min,0.000000e+00,-2.760000e+01
25%,5.750000e+00,6.100000e+00
50%,1.150000e+01,1.500000e+01
75%,1.725000e+01,2.240000e+01
max,2.300000e+01,4.260000e+01


### 1.3.2 시군구별 groupby

강수와 동일하게 기온도 중간값을 사용하였다.

In [ ]:
temp1_median = temp1.groupby(['광역시도','시군구','날짜','시간대'])['기온'].median().reset_index()
temp1_median

,광역시도,시군구,날짜,시간대,기온
0,강원도,강릉시,2019-08-01,0,30.900000
1,강원도,강릉시,2019-08-01,1,32.099998
2,강원도,강릉시,2019-08-01,2,32.099998
3,강원도,강릉시,2019-08-01,3,34.099998
4,강원도,강릉시,2019-08-01,4,34.099998
...,...,...,...,...,...
4607803,충청북도,충주시,2021-08-31,19,20.500000
4607804,충청북도,충주시,2021-08-31,20,20.000000
4607805,충청북도,충주시,2021-08-31,21,19.799999
4607806,충청북도,충주시,2021-08-31,22,19.799999


In [ ]:
temp2_median= temp2.groupby(['광역시도','시군구','날짜','시간대'])['기온'].median().reset_index()
temp2_median

,광역시도,시군구,날짜,시간대,기온
0,강원도,강릉시,2019-08-01,0,30.900000
1,강원도,강릉시,2019-08-01,1,32.099998
2,강원도,강릉시,2019-08-01,2,32.099998
3,강원도,강릉시,2019-08-01,3,34.099998
4,강원도,강릉시,2019-08-01,4,34.099998
...,...,...,...,...,...
4609315,충청북도,충주시,2021-08-31,19,20.300000
4609316,충청북도,충주시,2021-08-31,20,19.750000
4609317,충청북도,충주시,2021-08-31,21,19.649999
4609318,충청북도,충주시,2021-08-31,22,19.700000


In [ ]:
temp1_median.describe()

,시간대,기온
count,4.607808e+06,4.603801e+06
mean,1.150000e+01,1.395481e+01
std,6.922187e+00,1.007792e+01
min,0.000000e+00,-2.470000e+01
25%,5.750000e+00,6.100000e+00
50%,1.150000e+01,1.490000e+01
75%,1.725000e+01,2.230000e+01
max,2.300000e+01,3.820000e+01


최소값과 가까운 값의 데이터를 살펴보았다. 추운계절의 추운지역의 늦은시간의 기온이다. **이상값 아니다.**

In [ ]:
temp1_median[(temp1_median['기온']<-24)]

,광역시도,시군구,날짜,시간대,기온
1118662,경기도,하남시,2021-01-07,22,-24.700001
1118684,경기도,하남시,2021-01-08,20,-24.400000
1118685,경기도,하남시,2021-01-08,21,-24.500000
1118686,경기도,하남시,2021-01-08,22,-24.200001
3843570,전라북도,무주군,2021-01-07,18,-24.400000
3843571,전라북도,무주군,2021-01-07,19,-24.200001
3843573,전라북도,무주군,2021-01-07,21,-24.400000
3843574,전라북도,무주군,2021-01-07,22,-24.600000
4026454,전라북도,진안군,2021-01-07,22,-24.100000


In [ ]:
temp2_median.describe()

,시간대,기온
count,4.609320e+06,4.605320e+06
mean,1.150000e+01,1.394865e+01
std,6.922187e+00,1.006993e+01
min,0.000000e+00,-2.720000e+01
25%,5.750000e+00,6.100000e+00
50%,1.150000e+01,1.490000e+01
75%,1.725000e+01,2.230000e+01
max,2.300000e+01,3.820000e+01


추운지역의 늦은 가을날씨, 늦은 시간대, 연속적인 시간대 이므로 이상값이 아니다.

In [ ]:
temp1_median[(temp1_median['기온']<-3) & (temp1_median['날짜'].dt.month >=5) & (temp1_median['날짜'].dt.month <=10)]

,광역시도,시군구,날짜,시간대,기온
102233,강원도,양구군,2020-10-23,17,-3.1
102235,강원도,양구군,2020-10-23,19,-3.3
102354,강원도,양구군,2020-10-28,18,-3.4
102355,강원도,양구군,2020-10-28,19,-4.8
102356,강원도,양구군,2020-10-28,20,-4.7
102357,강원도,양구군,2020-10-28,21,-4.1
102378,강원도,양구군,2020-10-29,18,-4.0
102379,강원도,양구군,2020-10-29,19,-4.6
102380,강원도,양구군,2020-10-29,20,-3.9
102381,강원도,양구군,2020-10-29,21,-4.1


In [ ]:
temp2_median[(temp2_median['기온']<-5) & (temp2_median['날짜'].dt.month >=5) & (temp2_median['날짜'].dt.month <=10)]

,광역시도,시군구,날짜,시간대,기온


In [ ]:
temp_median = pd.concat([temp1_median,temp2_median], ignore_index=True)
temp_median

,광역시도,시군구,날짜,시간대,기온
0,강원도,강릉시,2019-08-01,0,30.900000
1,강원도,강릉시,2019-08-01,1,32.099998
2,강원도,강릉시,2019-08-01,2,32.099998
3,강원도,강릉시,2019-08-01,3,34.099998
4,강원도,강릉시,2019-08-01,4,34.099998
...,...,...,...,...,...
9217123,충청북도,충주시,2021-08-31,19,20.300000
9217124,충청북도,충주시,2021-08-31,20,19.750000
9217125,충청북도,충주시,2021-08-31,21,19.649999
9217126,충청북도,충주시,2021-08-31,22,19.700000


In [ ]:
temp_median= temp_median.groupby(['광역시도','시군구','날짜','시간대'])['기온'].median().reset_index()
temp_median

,광역시도,시군구,날짜,시간대,기온
0,강원도,강릉시,2019-08-01,0,30.900000
1,강원도,강릉시,2019-08-01,1,32.099998
2,강원도,강릉시,2019-08-01,2,32.099998
3,강원도,강릉시,2019-08-01,3,34.099998
4,강원도,강릉시,2019-08-01,4,34.099998
...,...,...,...,...,...
4626859,충청북도,충주시,2021-08-31,19,20.400000
4626860,충청북도,충주시,2021-08-31,20,19.875000
4626861,충청북도,충주시,2021-08-31,21,19.724999
4626862,충청북도,충주시,2021-08-31,22,19.749999


In [ ]:
temp_median[(temp_median['날짜'].dt.month >=6) & (temp_median['날짜'].dt.month <=10)].describe()

,시간대,기온
count,2.046264e+06,2.044623e+06
mean,1.150000e+01,2.216916e+01
std,6.922188e+00,5.614396e+00
min,0.000000e+00,-3.550000e+00
25%,5.750000e+00,1.920000e+01
50%,1.150000e+01,2.270000e+01
75%,1.725000e+01,2.590000e+01
max,2.300000e+01,3.820000e+01


In [ ]:
#배달데이터는 세종특별자치시의 시군구가 존재하지 않음
temp_median['시군구'] = np.where(temp_median['시군구']=='세종특별자치시', '\\N', temp_median['시군구'])
# 창원시를 제외한 ~시~구 형태의 시군구는 ~시 ~구로 변환(띄어쓰기 필요)
temp_median['시군구'] = np.where(temp_median['시군구'].str.endswith('구') & temp_median['시군구'].str.contains('시') & ~temp_median['시군구'].str.startswith('창원'),
                        temp_median['시군구'].str.replace('시',"시 "), temp_median['시군구'])
temp_median[temp_median['시군구']=='천안시 서북구']

,광역시도,시군구,날짜,시간대,기온
4297680,충청남도,천안시 서북구,2019-08-01,0,28.799999
4297681,충청남도,천안시 서북구,2019-08-01,1,30.700001
4297682,충청남도,천안시 서북구,2019-08-01,2,31.200001
4297683,충청남도,천안시 서북구,2019-08-01,3,30.299999
4297684,충청남도,천안시 서북구,2019-08-01,4,32.099998
...,...,...,...,...,...
4315963,충청남도,천안시 서북구,2021-08-31,19,20.700001
4315964,충청남도,천안시 서북구,2021-08-31,20,21.000000
4315965,충청남도,천안시 서북구,2021-08-31,21,20.299999
4315966,충청남도,천안시 서북구,2021-08-31,22,20.400000


In [ ]:
temp_median.to_csv('/content/drive/MyDrive/시군구_시간대별_중간기온.csv',index=False, encoding="utf-8-sig")